## **Applied Data Science Final Project**

##### Megha Polavarapu (mp4392), Xiaoying Wang (xw2993), Iris June Chang (ijc2119), Mengyan Li (ml4779)

### Data Acquisition and Cleaning

In [3]:
import os
import pandas as pd
import requests
import json
import time
import dateutil
import datetime
from dateutil.relativedelta import relativedelta
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [90]:
# pip install fastparquet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 685.4/685.4 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 17.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [ ]:
## might need to unzip/rezip raw data to upload to github bc of file size limitations 
# !unzip data/raw.zip

#### **Presidential Election Data 2020 by Precinct**
Data set link: https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/NT66Z3
Citation: MIT Election Data and Science Lab, 2022, "Precinct-Level Returns 2020 by Individual State", https://doi.org/10.7910/DVN/NT66Z3, Harvard Dataverse, V6, UNF:6:aViWPnsxmDD+s1GuFrrdpA== [fileUNF]

In [511]:
PATH_NAME = 'data/raw/precinct_election_data/'

filenames = next(os.walk(PATH_NAME), (None, None, []))[2]
filenames.remove('README.md')
filenames.remove('2020-precincts-codebook.md')
filenames.remove('.Rhistory')
print(filenames)
print(len(filenames))

['2020-la-precinct-general.csv', '2020-tx-precinct-general.csv', '2020-nd-precinct-general.csv', '2020-ky-precinct-general.csv', '2020-mi-precinct-general.csv', '2020-ks-precinct-general.csv', '2020-wa-precinct-general.csv', '2020-or-precinct-general.csv', '2020-ms-precinct-general.csv', '2020-de-precinct-general.csv', '2020-ri-precinct-general.csv', '2020-oh-precinct-general.csv', '2020-ar-precinct-general.csv', '2020-al-precinct-general.csv', '2020-ut-precinct-general.csv', '2020-nj-precinct-general.csv', '2020-sd-precinct-general.csv', '2020-il-precinct-general.csv', '2020-ne-precinct-general.csv', '2020-nc-precinct-general-sorted.csv', '2020-ct-precinct-general.csv', '2020-nc-precinct-general.csv', '2020-mn-precinct-general.csv', '2020-fl-precinct-general.csv', '2020-wi-precinct-general.csv', '2020-ma-precinct-general.csv', '2020-tn-precinct-general.csv', '2020-md-precinct-general.csv', '2020-va-precinct-general.csv', '2020-ok-precinct-general.csv', '2020-ga-precinct-general.csv', 

Based on there being 52 file names, we can tell there is a duplicate (North Carolina)

In [515]:
file = 'data/raw/precinct_election_data/2020-nc-precinct-general.csv'
nc = pd.read_csv(file)
file = 'data/raw/precinct_election_data/2020-nc-precinct-general-sorted.csv'
nc_sorted = pd.read_csv(file)

print(nc.shape[0])
print(nc_sorted.shape[0])

filenames.remove('2020-nc-precinct-general-sorted.csv')

/var/folders/86/wprmb_s52rs0t4r2gfzh2df00000gn/T/ipykernel_48888/797725649.py:2: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  nc = pd.read_csv(file)
/var/folders/86/wprmb_s52rs0t4r2gfzh2df00000gn/T/ipykernel_48888/797725649.py:4: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  nc_sorted = pd.read_csv(file)


696804
1529852


We choose to keep the sorted data because it has more data. Based on the readMe for this dataset, contains additional information about the ballots used which could be used and therefore, this dataset will be used. 

In [518]:
# file = 'data/raw/precinct_election_data/2020-wv-precinct-general.csv'
combined = pd.DataFrame()
for file in filenames:
    print('Loading file ', file)
    d = pd.read_csv(PATH_NAME + file)
            
    # standardizing formatting in precinct column 
    d['precinct'] = d['precinct'].replace('PRECINCT ', '')
    d['votes'] = d['votes'].astype(int)
    
    # only keeping presidential election data 
    df = d.groupby(['candidate','office', 'party_detailed',	'party_simplified',	'state','state_po'], as_index=False)['votes'].sum()
    
    # re adding relevant data 
    # df = df.drop(columns = ['precinct', 'mode', 
    #        'county_name', 'county_fips', 'jurisdiction_name',
    #        'jurisdiction_fips', 'district', 'magnitude', 'dataverse',
    #        'year', 'stage', 'special', 'writein', 'state_fips', 'state_cen', 'state_ic', 'date', 'readme_check'])
    # df = df.merge(d, left_on='candidate', right_on='candidate', how='left')
    # df = df.drop_duplicates().reset_index()
    # df= df.drop(columns = ['index'])
    
    combined = pd.concat([combined,df], axis=0)

Loading file  2020-la-precinct-general.csv
Loading file  2020-tx-precinct-general.csv


/var/folders/86/wprmb_s52rs0t4r2gfzh2df00000gn/T/ipykernel_48888/3461128695.py:5: DtypeWarning: Columns (2,3,10,11,13) have mixed types. Specify dtype option on import or set low_memory=False.
  d = pd.read_csv(PATH_NAME + file)


Loading file  2020-nd-precinct-general.csv
Loading file  2020-ky-precinct-general.csv
Loading file  2020-mi-precinct-general.csv
Loading file  2020-ks-precinct-general.csv
Loading file  2020-wa-precinct-general.csv


/var/folders/86/wprmb_s52rs0t4r2gfzh2df00000gn/T/ipykernel_48888/3461128695.py:5: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  d = pd.read_csv(PATH_NAME + file)


Loading file  2020-or-precinct-general.csv
Loading file  2020-ms-precinct-general.csv
Loading file  2020-de-precinct-general.csv
Loading file  2020-ri-precinct-general.csv
Loading file  2020-oh-precinct-general.csv


...

/var/folders/86/wprmb_s52rs0t4r2gfzh2df00000gn/T/ipykernel_48888/3461128695.py:5: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  d = pd.read_csv(PATH_NAME + file)


Loading file  2020-pa-precinct-general.csv
Loading file  2020-mo-precinct-general.csv
Loading file  2020-az-precinct-general.csv
Loading file  2020-ny-precinct-general.csv


/var/folders/86/wprmb_s52rs0t4r2gfzh2df00000gn/T/ipykernel_48888/3461128695.py:5: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  d = pd.read_csv(PATH_NAME + file)


Loading file  2020-wv-precinct-general.csv
Loading file  2020-hi-precinct-general.csv
Loading file  2020-mt-precinct-general.csv
Loading file  2020-ca-precinct-general.csv
Loading file  2020-nv-precinct-general.csv
Loading file  2020-wy-precinct-general.csv


/var/folders/86/wprmb_s52rs0t4r2gfzh2df00000gn/T/ipykernel_48888/3461128695.py:5: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  d = pd.read_csv(PATH_NAME + file)


In [522]:
combined = combined.map(lambda x: x.lower() if isinstance(x, str) else x)

In [524]:
combined.dtypes

candidate           object
office              object
party_detailed      object
party_simplified    object
state               object
state_po            object
votes                int64
dtype: object

In [526]:
print(combined.head())
print([combined[col_name].unique() for col_name in combined.columns if col_name == 'state'])

           candidate                office party_detailed party_simplified  \
0     aaron c sigler             us senate    libertarian      libertarian   
1  aaron chaisson jr  district court judge     republican       republican   
2     adrian perkins             us senate       democrat         democrat   
3        alicia butl  district court judge       democrat         democrat   
4  allen guillory sr              us house     republican       republican   

       state state_po   votes  
0  louisiana       la   11321  
1  louisiana       la   30725  
2  louisiana       la  394049  
3  louisiana       la    5038  
4  louisiana       la   22496  
[array(['louisiana', 'texas', 'north dakota', 'kentucky', 'michigan',
       'kansas', 'washington', 'oregon', 'mississippi', 'delaware',
       'rhode island', 'ohio', 'arkansas', 'alabama', 'utah',
       'new jersey', 'south dakota', 'illinois', 'nebraska',
       'connecticut', 'north carolina', 'minnesota', 'florida',
       'wiscon

In [528]:
combined.to_csv('data/2020_election_vote_counts.csv', index=False, encoding='utf-8-sig')

In [530]:
## Getting who won each office by state
top_candidates = combined.loc[combined.groupby(['office', 'state'])['votes'].idxmax()].reset_index(drop=True)
top_candidates = top_candidates.sort_values('state')

In [532]:
top_candidates

,candidate,office,party_detailed,party_simplified,state,state_po,votes
22835,mary windom,court of criminal appeals judge,republican,republican,alabama,al,1541862
22537,joeletta martin barrentine,circuit court judge,republican,republican,alabama,al,26536
19299,tonya smith chestnut,state board of education,democrat,democrat,alabama,al,161192
10242,terri a sewell,us house,democrat,democrat,alabama,al,225742
22697,donald j trump,us president,republican,republican,alabama,al,1441170
...,...,...,...,...,...,...,...
8275,donald j trump,us president,republican,republican,wyoming,wy,193559
7836,bill henderson,state house,republican,republican,wyoming,wy,2497
18228,levi j shinkle,state house,democrat,democrat,wyoming,wy,816
15575,ocean andrew,state house,republican,republican,wyoming,wy,3409


In [534]:
top_candidates.to_csv('data/2020_election_vote_results.csv', index=False, encoding='utf-8-sig')

#### **2020 Census Data**
Data source link: https://www.census.gov/data/tables/2023/dec/2020-census-demographic-profile.html

Citation: U.S. Census Bureau, U.S. Department of Commerce. "PROFILE OF GENERAL POPULATION AND HOUSING CHARACTERISTICS." Decennial Census, DEC Demographic Profile, Table DP1, 2020, https://data.census.gov/table/DECENNIALDP2020.DP1?g=040XX00US01,02,04,05,06,08,09,10,11,12,13,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,44,45,46,47,48,49,50,51,53,54,55,56,72&d=DEC+Demographic+Profile&tp=true. 

In [566]:
df = pd.read_csv('data/raw/census_data/census_raw.csv')
df.head(10)
# can see that the data is quite messy 

,GEO_ID,NAME,DP1_0001C,DP1_0002C,DP1_0003C,DP1_0004C,DP1_0005C,DP1_0006C,DP1_0007C,DP1_0008C,...,DP1_0152P,DP1_0153P,DP1_0154P,DP1_0155P,DP1_0156P,DP1_0157P,DP1_0158P,DP1_0159P,DP1_0160P,Unnamed: 322
0,Geography,Geographic Area Name,Count!!SEX AND AGE!!Total population,Count!!SEX AND AGE!!Total population!!Under 5 ...,Count!!SEX AND AGE!!Total population!!5 to 9 y...,Count!!SEX AND AGE!!Total population!!10 to 14...,Count!!SEX AND AGE!!Total population!!15 to 19...,Count!!SEX AND AGE!!Total population!!20 to 24...,Count!!SEX AND AGE!!Total population!!25 to 29...,Count!!SEX AND AGE!!Total population!!30 to 34...,...,Percent!!HOUSING OCCUPANCY!!Total housing unit...,Percent!!HOUSING OCCUPANCY!!Total housing unit...,Percent!!HOUSING OCCUPANCY!!Total housing unit...,Percent!!HOUSING OCCUPANCY!!Total housing unit...,Percent!!VACANCY RATES!!Homeowner vacancy rate...,Percent!!VACANCY RATES!!Rental vacancy rate (p...,Percent!!HOUSING TENURE!!Occupied housing units,Percent!!HOUSING TENURE!!Occupied housing unit...,Percent!!HOUSING TENURE!!Occupied housing unit...,NaN
1,0400000US01,Alabama,5024279,286529,302637,325031,338475,345931,314244,311116,...,1.0,0.5,2.5,4.6,(X),(X),100.0,67.7,32.3,NaN
2,0400000US02,Alaska,733391,48104,51054,51344,47433,49456,55058,56981,...,1.0,0.4,9.1,3.6,(X),(X),100.0,63.9,36.1,NaN
3,0400000US04,Arizona,7151502,392370,443878,485297,485891,477713,473578,462909,...,1.0,0.4,5.9,1.7,(X),(X),100.0,65.3,34.7,NaN
4,0400000US05,Arkansas,3011524,179575,192794,205837,204915,198109,188836,190366,...,1.1,0.7,2.4,4.3,(X),(X),100.0,65.0,35.0,NaN
5,0400000US06,California,39538223,2137439,2393219,2613891,2644071,2731553,2915258,2911574,...,0.5,0.2,2.1,1.3,(X),(X),100.0,54.5,45.5,NaN
6,0400000US08,Colorado,5773714,314580,347525,377510,375364,382920,444668,451164,...,0.7,0.3,4.2,1.4,(X),(X),100.0,64.5,35.5,NaN
7,0400000US09,Connecticut,3605944,176831,197636,221199,240933,235047,222207,232108,...,0.8,0.3,1.8,1.8,(X),(X),100.0,64.2,35.8,NaN
8,0400000US10,Delaware,989948,51230,56920,61324,62659,65475,60431,62234,...,0.9,0.3,8.1,2.0,(X),(X),100.0,70.8,29.2,NaN
9,0400000US11,District of Columbia,689545,37068,33308,28928,33747,64439,83277,79300,...,0.8,0.4,0.8,2.5,(X),(X),100.0,38.3,61.7,NaN


In [568]:
headers = df.iloc[0]
df  = pd.DataFrame(df.values[1:], columns=headers)

In [570]:
df.head(10)

,Geography,Geographic Area Name,Count!!SEX AND AGE!!Total population,Count!!SEX AND AGE!!Total population!!Under 5 years,Count!!SEX AND AGE!!Total population!!5 to 9 years,Count!!SEX AND AGE!!Total population!!10 to 14 years,Count!!SEX AND AGE!!Total population!!15 to 19 years,Count!!SEX AND AGE!!Total population!!20 to 24 years,Count!!SEX AND AGE!!Total population!!25 to 29 years,Count!!SEX AND AGE!!Total population!!30 to 34 years,...,Percent!!HOUSING OCCUPANCY!!Total housing units!!Vacant housing units!!For sale only,"Percent!!HOUSING OCCUPANCY!!Total housing units!!Vacant housing units!!Sold, not occupied","Percent!!HOUSING OCCUPANCY!!Total housing units!!Vacant housing units!!For seasonal, recreational, or occasional use",Percent!!HOUSING OCCUPANCY!!Total housing units!!Vacant housing units!!All other vacants,Percent!!VACANCY RATES!!Homeowner vacancy rate (percent) [4],Percent!!VACANCY RATES!!Rental vacancy rate (percent) [5],Percent!!HOUSING TENURE!!Occupied housing units,Percent!!HOUSING TENURE!!Occupied housing units!!Owner-occupied housing units,Percent!!HOUSING TENURE!!Occupied housing units!!Renter-occupied housing units,NaN
0,0400000US01,Alabama,5024279,286529,302637,325031,338475,345931,314244,311116,...,1.0,0.5,2.5,4.6,(X),(X),100.0,67.7,32.3,NaN
1,0400000US02,Alaska,733391,48104,51054,51344,47433,49456,55058,56981,...,1.0,0.4,9.1,3.6,(X),(X),100.0,63.9,36.1,NaN
2,0400000US04,Arizona,7151502,392370,443878,485297,485891,477713,473578,462909,...,1.0,0.4,5.9,1.7,(X),(X),100.0,65.3,34.7,NaN
3,0400000US05,Arkansas,3011524,179575,192794,205837,204915,198109,188836,190366,...,1.1,0.7,2.4,4.3,(X),(X),100.0,65.0,35.0,NaN
4,0400000US06,California,39538223,2137439,2393219,2613891,2644071,2731553,2915258,2911574,...,0.5,0.2,2.1,1.3,(X),(X),100.0,54.5,45.5,NaN
5,0400000US08,Colorado,5773714,314580,347525,377510,375364,382920,444668,451164,...,0.7,0.3,4.2,1.4,(X),(X),100.0,64.5,35.5,NaN
6,0400000US09,Connecticut,3605944,176831,197636,221199,240933,235047,222207,232108,...,0.8,0.3,1.8,1.8,(X),(X),100.0,64.2,35.8,NaN
7,0400000US10,Delaware,989948,51230,56920,61324,62659,65475,60431,62234,...,0.9,0.3,8.1,2.0,(X),(X),100.0,70.8,29.2,NaN
8,0400000US11,District of Columbia,689545,37068,33308,28928,33747,64439,83277,79300,...,0.8,0.4,0.8,2.5,(X),(X),100.0,38.3,61.7,NaN
9,0400000US12,Florida,21538187,1030284,1151858,1262516,1289224,1285788,1291440,1348166,...,1.2,0.4,6.8,1.8,(X),(X),100.0,65.1,34.9,NaN


In [572]:
import re

new_names = df.columns.tolist()
new_names = [str(col) for col in new_names]

replace = ['Count', 'SEX AND AGE','TOTAL RACES TALLIED','HISPANIC OR LATINO BY RACE','RELATIONSHIP',
           'HOUSEHOLDS BY TYPE','HOUSING OCCUPANCY','VACANCY RATES','HOUSING TENURE','RACE', 'BY SEX '] 

for i, name in enumerate(new_names):
    n = name.replace('!!', ' ')
    n = n.replace('  ', ' ')
    n = re.sub(r'\[\d+\]\s*', '', n)
    if name == 'Geographic Area Name':
        n = n.replace('Geographic Area Name', 'State')
    
    for word in replace:
        if word in n:
            n = n.replace(word,'')
    n = n.lower()
    
    new_names[i] = n.strip()
            

In [574]:
new_names
# we want to remove redundant columns (selected age categories

['geography',
 'state',
 'total population',
 'total population under 5 years',
 'total population 5 to 9 years',
 'total population 10 to 14 years',
 'total population 15 to 19 years',
 'total population 20 to 24 years',
 'total population 25 to 29 years',
 'total population 30 to 34 years',
 'total population 35 to 39 years',
 'total population 40 to 44 years',
 'total population 45 to 49 years',
 'total population 50 to 54 years',
 'total population 55 to 59 years',
 'total population 60 to 64 years',
 'total population 65 to 69 years',
 'total population 70 to 74 years',
 'total population 75 to 79 years',
 'total population 80 to 84 years',
 'total population 85 years and over',
 'total population selected age categories 16 years and over',
 'total population selected age categories 18 years and over',
 'total population selected age categories 21 years and over',
 'total population selected age categories 62 years and over',
 'total population selected age categories 65 years and

In [576]:
# reassign column names to new_names
df  = pd.DataFrame(df.values[:], columns=new_names)
df.head()

,geography,state,total population,total population under 5 years,total population 5 to 9 years,total population 10 to 14 years,total population 15 to 19 years,total population 20 to 24 years,total population 25 to 29 years,total population 30 to 34 years,...,percent total housing units vacant housing units for sale only,"percent total housing units vacant housing units sold, not occupied","percent total housing units vacant housing units for seasonal, recreational, or occasional use",percent total housing units vacant housing units all other vacants,percent homeowner vacancy rate (percent),percent rental vacancy rate (percent),percent occupied housing units,percent occupied housing units owner-occupied housing units,percent occupied housing units renter-occupied housing units,nan
0,0400000US01,Alabama,5024279,286529,302637,325031,338475,345931,314244,311116,...,1.0,0.5,2.5,4.6,(X),(X),100.0,67.7,32.3,NaN
1,0400000US02,Alaska,733391,48104,51054,51344,47433,49456,55058,56981,...,1.0,0.4,9.1,3.6,(X),(X),100.0,63.9,36.1,NaN
2,0400000US04,Arizona,7151502,392370,443878,485297,485891,477713,473578,462909,...,1.0,0.4,5.9,1.7,(X),(X),100.0,65.3,34.7,NaN
3,0400000US05,Arkansas,3011524,179575,192794,205837,204915,198109,188836,190366,...,1.1,0.7,2.4,4.3,(X),(X),100.0,65.0,35.0,NaN
4,0400000US06,California,39538223,2137439,2393219,2613891,2644071,2731553,2915258,2911574,...,0.5,0.2,2.1,1.3,(X),(X),100.0,54.5,45.5,NaN


In [578]:
col_to_drop = []
col_to_drop.append('nan')
col_to_drop.append('geography')

for col in new_names:
    if 'selected age categories' in col:
        col_to_drop.append(col) 


In [580]:
col_to_drop

['nan',
 'geography',
 'total population selected age categories 16 years and over',
 'total population selected age categories 18 years and over',
 'total population selected age categories 21 years and over',
 'total population selected age categories 62 years and over',
 'total population selected age categories 65 years and over',
 'male population selected age categories 16 years and over',
 'male population selected age categories 18 years and over',
 'male population selected age categories 21 years and over',
 'male population selected age categories 62 years and over',
 'male population selected age categories 65 years and over',
 'female population selected age categories 16 years and over',
 'female population selected age categories 18 years and over',
 'female population selected age categories 21 years and over',
 'female population selected age categories 62 years and over',
 'female population selected age categories 65 years and over',
 'percent  total population selec

In [582]:
df_final = df.copy(deep=True)
for col in col_to_drop:
    if col in df.columns.tolist():
        df_final = df_final.drop(col, axis=1)

In [584]:
# remove duplicate columns
df_final = df_final.loc[:,~df_final.columns.duplicated()].copy()

In [586]:
df_final.columns.tolist()

['state',
 'total population',
 'total population under 5 years',
 'total population 5 to 9 years',
 'total population 10 to 14 years',
 'total population 15 to 19 years',
 'total population 20 to 24 years',
 'total population 25 to 29 years',
 'total population 30 to 34 years',
 'total population 35 to 39 years',
 'total population 40 to 44 years',
 'total population 45 to 49 years',
 'total population 50 to 54 years',
 'total population 55 to 59 years',
 'total population 60 to 64 years',
 'total population 65 to 69 years',
 'total population 70 to 74 years',
 'total population 75 to 79 years',
 'total population 80 to 84 years',
 'total population 85 years and over',
 'male population',
 'male population under 5 years',
 'male population 5 to 9 years',
 'male population 10 to 14 years',
 'male population 15 to 19 years',
 'male population 20 to 24 years',
 'male population 25 to 29 years',
 'male population 30 to 34 years',
 'male population 35 to 39 years',
 'male population 40 to 

In [610]:
# removing Puerto Rico because they are not represented in Congress or electoral college 
df_final = df_final[df_final.state != 'Puerto Rico']

In [612]:
df_final['state'].unique

<bound method Series.unique of 0                  Alabama
1                   Alaska
2                  Arizona
3                 Arkansas
4               California
5                 Colorado
6              Connecticut
7                 Delaware
8     District of Columbia
9                  Florida
10                 Georgia
11                  Hawaii
12                   Idaho
13                Illinois
14                 Indiana
15                    Iowa
16                  Kansas
17                Kentucky
18               Louisiana
19                   Maine
20                Maryland
21           Massachusetts
22                Michigan
23               Minnesota
24             Mississippi
25                Missouri
26                 Montana
27                Nebraska
28                  Nevada
29           New Hampshire
30              New Jersey
31              New Mexico
32                New York
33          North Carolina
34            North Dakota
35                    Oh

In [624]:
#making all data all lowercase
df_final = df_final.map(lambda x: x.lower() if isinstance(x, str) else x)

In [626]:
df_final.to_csv("data/census_data", index=False, encoding='utf-8-sig')

#### **Combining Census and Election Data**

In [628]:
census = pd.read_csv("data/census_data") 
election = pd.read_csv("data/2020_election_vote_counts.csv") 

combined = election.merge(census, left_on='state', right_on='state', how = 'left')
combined.head()

,candidate,office,party_detailed,party_simplified,state,state_po,votes,total population,total population under 5 years,total population 5 to 9 years,...,"percent total housing units vacant housing units rented, not occupied",percent total housing units vacant housing units for sale only,"percent total housing units vacant housing units sold, not occupied","percent total housing units vacant housing units for seasonal, recreational, or occasional use",percent total housing units vacant housing units all other vacants,percent homeowner vacancy rate (percent),percent rental vacancy rate (percent),percent occupied housing units,percent occupied housing units owner-occupied housing units,percent occupied housing units renter-occupied housing units
0,aaron c sigler,us senate,libertarian,libertarian,louisiana,la,11321,4657757,281257,300637,...,0.3,1.0,0.4,1.6,4.8,(x),(x),100.0,65.5,34.5
1,aaron chaisson jr,district court judge,republican,republican,louisiana,la,30725,4657757,281257,300637,...,0.3,1.0,0.4,1.6,4.8,(x),(x),100.0,65.5,34.5
2,adrian perkins,us senate,democrat,democrat,louisiana,la,394049,4657757,281257,300637,...,0.3,1.0,0.4,1.6,4.8,(x),(x),100.0,65.5,34.5
3,alicia butl,district court judge,democrat,democrat,louisiana,la,5038,4657757,281257,300637,...,0.3,1.0,0.4,1.6,4.8,(x),(x),100.0,65.5,34.5
4,allen guillory sr,us house,republican,republican,louisiana,la,22496,4657757,281257,300637,...,0.3,1.0,0.4,1.6,4.8,(x),(x),100.0,65.5,34.5


In [638]:
#There are no null values
combined[combined.isnull().any(axis=1)]

,candidate,office,party_detailed,party_simplified,state,state_po,votes,total population,total population under 5 years,total population 5 to 9 years,...,"percent total housing units vacant housing units rented, not occupied",percent total housing units vacant housing units for sale only,"percent total housing units vacant housing units sold, not occupied","percent total housing units vacant housing units for seasonal, recreational, or occasional use",percent total housing units vacant housing units all other vacants,percent homeowner vacancy rate (percent),percent rental vacancy rate (percent),percent occupied housing units,percent occupied housing units owner-occupied housing units,percent occupied housing units renter-occupied housing units


In [640]:
combined = combined.drop(['state_po'], axis = 1)

In [642]:
df_final.to_csv("data/election_census_combined", index=False, encoding='utf-8-sig')

#### **2020 NYT Article Titles**
##### References:    
- https://github.com/susannapaoli/web-scraper-nyt/tree/main    
- https://developer.nytimes.com/get-started    
- 

In [304]:
# commented out bc I revoked this API for security reasons
API_KEY = None
#'bgsBWGVUUDgyvMaTcXZZy4yd9Ogsehqh'

In [314]:
def parse_response(response):
    data = {'headline': [],  
        'date': [], 
        'web_url': [],
        'doc_type': [],
        'lead_paragraph': [],
        'material_type': [],
        'author': [],
        'section': [],
        'subsection': [],
        'keywords': []}

    articles = response['response']['docs'] 
    for article in articles: 
        date = dateutil.parser.parse(article['pub_date']).date()
        # if date != datetime.date(2020, 1, 1):
        #     continue
        if type(article['headline']) == dict and 'main' in article['headline'].keys():
            data['date'].append(date)
            data['headline'].append(article['headline']['main']) 
            if 'section_name' in article:
                data['section'].append(article['section_name'])
            else:
                data['section'].append(None)
            if 'lead_paragraph' in article:
                data['lead_paragraph'].append(article['lead_paragraph'])
            else:
                data['lead_paragraph'].append(None)
            if 'web_url' in article:
                data['web_url'].append(article['web_url'])
            else:
                data['web_url'].append(None)
            if 'subsection_name' in article:
                data['subsection'].append(article['subsection_name'])
            else:
                data['subsection'].append(None)
            if 'byline' in article:
                data['author'].append(article['byline']['original'])
            else:
                data['author'].append(None)
            data['doc_type'].append(article['document_type'])
            if 'type_of_material' in article: 
                data['material_type'].append(article['type_of_material'])
            else:
                data['material_type'].append(None)
            keywords = [keyword['value'] for keyword in article['keywords'] if keyword['name'] == 'subject']
            data['keywords'].append(keywords)
    return pd.DataFrame(data) 

In [114]:
# Get data from January 2020 to May 2020 
for i in range(1,6): 
    date = ['2020', str(i)]
    print('Working on', date)
    base_url = 'https://api.nytimes.com/svc/archive/v1/'
    url = base_url + '/' + date[0] + '/' + date[1] + '.json?api-key=' + API_KEY
    response = requests.get(url, verify=False).json()
    time.sleep(6) 
    
    df = parse_response(response)
    csv_path = 'data/raw/nyt_headlines/' + date[0] + '-' + date[1] + '.csv'
    df.to_csv(csv_path, index=False)

Working on ['2020', '1']
Working on ['2020', '2']
Working on ['2020', '3']
Working on ['2020', '4']
Working on ['2020', '5']


In [19]:
# Get data from June 2020 to November 2020 
for i in range(6,12): 
    date = ['2020', str(i)]
    print('Working on', date)
    base_url = 'https://api.nytimes.com/svc/archive/v1/'
    url = base_url + '/' + date[0] + '/' + date[1] + '.json?api-key=' + API_KEY
    response = requests.get(url, verify=False).json()
    time.sleep(6) 
    
    df = parse_response(response)
    csv_path = 'data/raw/nyt_headlines/' + date[0] + '-' + date[1] + '.csv'
    df.to_csv(csv_path, index=False)

Working on ['2020', '6']
Working on ['2020', '7']
Working on ['2020', '8']
Working on ['2020', '9']
Working on ['2020', '10']
Working on ['2020', '11']


In [672]:
from datetime import datetime
files = ['data/raw/nyt_headlines/2020-' + str(i) + '.csv' for i in range(1,12)]

df = pd.DataFrame()
for f in files:
    d = pd.read_csv(f)
    df = pd.concat([df,d], axis=0)

#making all data all lowercase
# df = df.map(lambda x: x.lower() if isinstance(x, str) else x)
df['date'] = pd.to_datetime(df['date'])
df = df[df['date'] < datetime(2020,11,4)]

#export to csv
df.to_csv("data/nyt_data.csv", index=False, encoding='utf-8-sig')

In [674]:
df['date'].tail

<bound method NDFrame.tail of 0      2020-01-01
1      2020-01-01
2      2020-01-01
3      2020-01-01
4      2020-01-01
          ...    
1195   2020-11-03
1196   2020-11-03
1197   2020-11-03
1198   2020-11-03
1199   2020-11-03
Name: date, Length: 47214, dtype: datetime64[ns]>

In [676]:
nyt = pd.read_csv('data/nyt_data.csv')

In [678]:
nyt.describe()

,headline,date,web_url,doc_type,lead_paragraph,material_type,author,section,subsection,keywords
count,47211,47214,47214,47214,46226,45968,41665,47213,20253,47214
unique,46037,307,47213,3,40192,19,9836,51,81,30578
top,Coronavirus Briefing: What Happened Today,2020-11-03,https://www.nytimes.com/2020/10/06/world/cuomo...,article,To the Editor:,News,By The New York Times,U.S.,Politics,[]
freq,167,901,2,43280,625,33384,537,10575,4547,4537


#### **2020 Fox News Article Titles**
##### Reference: 
- https://medium.com/analytics-vidhya/the-wayback-machine-scraper-63238f6abb66

In [708]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import dateutil
from datetime import timedelta, datetime
from urllib.parse import urljoin


def get_wayback(base_url, date_str):
    api_url = 'https://archive.org/wayback/available?url=' + base_url + '&timestamp='+ date_str
    response = requests.get(api_url)
    data = response.json()
    try:
        snapshot = data['archived_snapshots']['closest']['url']
        return snapshot
    except KeyError:
        return None

def get_title(snapshot_url, base_url):
    headers = {'User-Agent': 'Mozilla/5.0'}
    resp = requests.get(snapshot_url, headers=headers)
    soup = BeautifulSoup(resp.text, 'html.parser')

    articles = []
    # Fox News often uses <h2> or <h4> tags with links inside
    for tag in soup.find_all(['h2', 'h4']):
        a = tag.find('a')
        if a and a.text.strip():
            title = a.text.strip()
            href = a.get('href')
            if href and not href.startswith('http'):
                href = urljoin("https://web.archive.org", href)
            articles.append((title, href))
    return articles

    # Loop through all days in January 2020
def get_data(start_date, end_date):    
    base_url = "https://www.foxnews.com"
    data = []
    
    while start_date <= end_date:
        date_str = start_date.strftime('%Y%m%d')
        snapshot_url = get_wayback(base_url, date_str)
        
        if snapshot_url:
            try:
                articles = get_title(snapshot_url, base_url)
                for title, url in articles:
                    data.append({
                        "title": title,
                        "date": start_date.strftime('%Y-%m-%d')
                    })
            except Exception as e:
                print(f"Error on {date_str}: {e}")
        
        start_date += timedelta(days=1)
        time.sleep(1)  
    return data 

In [720]:
months = [(datetime(2020, 1, 1), datetime(2020, 1, 31)), (datetime(2020, 2, 1), datetime(2020, 2, 29)), (datetime(2020, 3, 1), datetime(2020, 3, 31)),
         (datetime(2020, 4, 1), datetime(2020, 4, 30)), (datetime(2020, 5, 1), datetime(2020, 5, 31)), (datetime(2020, 6, 1), datetime(2020, 6, 30)),
         (datetime(2020, 7, 1), datetime(2020, 7, 31)),(datetime(2020, 8, 1), datetime(2020, 8, 31)), (datetime(2020, 9, 1), datetime(2020, 9, 30)),
         (datetime(2020, 10, 1), datetime(2020, 10, 31)), (datetime(2020, 11, 1), datetime(2020, 11, 3))]



In [726]:
# Need to split to prevent time out accessing link 
# Jan 2020 to March 2020

for i, pair in enumerate(months[:3]):
    # Create DataFrame
    print('Loading data from ', pair)
    data = get_data(pair[0], pair[1])
    df = pd.DataFrame(data)
    print('Collected ', len(df), ' articles from month')

    csv_path = 'data/raw/fox_headlines/' + '2020-' + str(i+1)+ '.csv'
    df.to_csv(csv_path, index=False)

Loading data from  (datetime.datetime(2020, 1, 1, 0, 0), datetime.datetime(2020, 1, 31, 0, 0))
Collected  3651  articles from month
Loading data from  (datetime.datetime(2020, 2, 1, 0, 0), datetime.datetime(2020, 2, 29, 0, 0))
Collected  3309  articles from month
Loading data from  (datetime.datetime(2020, 3, 1, 0, 0), datetime.datetime(2020, 3, 31, 0, 0))
Collected  3234  articles from month


In [723]:
# April 2020 to May 2020

for i, pair in enumerate(months[3:5]):
    # Create DataFrame
    print('Loading data from ', pair)
    data = get_data(pair[0], pair[1])
    df = pd.DataFrame(data)
    print('Collected ', len(df), ' articles from month')

    csv_path = 'data/raw/fox_headlines/' + '2020-' + str(i+4)+ '.csv'
    df.to_csv(csv_path, index=False)

Loading data from  (datetime.datetime(2020, 4, 1, 0, 0), datetime.datetime(2020, 4, 30, 0, 0))
Collected  1767  articles from month
Loading data from  (datetime.datetime(2020, 5, 1, 0, 0), datetime.datetime(2020, 5, 31, 0, 0))
Collected  3682  articles from month


In [722]:
# June 2020 to Nov 2020
for i, pair in enumerate(months[5:]):
    # Create DataFrame
    print('Loading data from ', pair)
    data = get_data(pair[0], pair[1])
    df = pd.DataFrame(data)
    print('Collected ', len(df), ' articles from month')

    csv_path = 'data/raw/fox_headlines/' + '2020-' + str(i+6)+ '.csv'
    df.to_csv(csv_path, index=False)

Loading data from  (datetime.datetime(2020, 6, 1, 0, 0), datetime.datetime(2020, 6, 30, 0, 0))
Collected  2349  articles from month
Loading data from  (datetime.datetime(2020, 7, 1, 0, 0), datetime.datetime(2020, 7, 31, 0, 0))
Collected  2144  articles from month
Loading data from  (datetime.datetime(2020, 8, 1, 0, 0), datetime.datetime(2020, 8, 31, 0, 0))
Collected  2760  articles from month
Loading data from  (datetime.datetime(2020, 9, 1, 0, 0), datetime.datetime(2020, 9, 30, 0, 0))
Collected  1795  articles from month
Loading data from  (datetime.datetime(2020, 10, 1, 0, 0), datetime.datetime(2020, 10, 31, 0, 0))
Collected  3175  articles from month
Loading data from  (datetime.datetime(2020, 11, 1, 0, 0), datetime.datetime(2020, 11, 3, 0, 0))
Collected  400  articles from month


In [728]:
files = ['data/raw/fox_headlines/2020-' + str(i) + '.csv' for i in range(1,12)]

df = pd.DataFrame()
for f in files:
    d = pd.read_csv(f)
    df = pd.concat([df,d], axis=0)

#making all all lowercase
# df = df.map(lambda x: x.lower() if isinstance(x, str) else x)

#export to csv
df.to_csv("data/fox_data.csv", index=False, encoding='utf-8-sig')

Looking at this data, there are some days that are missing. Upon further exploration, I think it is because of limitations of the web scraping was set up. By going through wayback machine, it is missing some screenshots of the Fox site leading to some missing data